In [1]:
import os
import sys
import codecs
import collections

In [2]:
import nltk
from lxml import etree
from lxml.html.clean import Cleaner
from pymystem3 import Mystem

In [3]:
file = codecs.open('datasets/golos_io_topics.xml', encoding='utf-8')
data = file.read();

In [4]:
cleaner = Cleaner()
cleaner.allow_tags = ['item', 'field']
cleaner.remove_unknown_tags = False
clean_data = cleaner.clean_html(data)

In [5]:
fout = codecs.open('datasets/golos_io_topics-res.xml', encoding='utf-8', mode='w+')
fout.write(clean_data)
fout.close()

In [6]:
from nltk.corpus import stopwords
stopwords_list = stopwords.words('russian')

In [7]:
tree = etree.fromstring(clean_data)
items = tree.xpath("//field[@name='body']")

In [8]:
seq_list = [item.text for item in items]

In [9]:
mystem = Mystem()

def convert_prefix2utags(mystem_prefix):
    utags_dict = {'_A':       '_ADJ',
                  '_ADV':     '_ADV',                                                                                                                                                                                                                                                                    
                  '_ADVPRO':  '_ADV',                                                                                                                                                                                                                                                                    
                  '_ANUM':    '_ADJ',                                                                                                                                                                                                                                                                 
                  '_APRO':    '_DET',                                                                                                                                                                                                                                                                   
                  '_COM':     '_ADJ',                                                                                                                                                                                                                                                                    
                  '_CONJ':    '_SCONJ',                                                                                                                                                                                                                                                                  
                  '_INTJ':    '_INTJ',                                                                                                                                                                                                                                                                   
                  '_NONLEX':  '_X',                                                                                                                                                                                                                                                                      
                  '_NUM':     '_NUM',                                                                                                                                                                                                                                                                    
                  '_PART':    '_PART',                                                                                                                                                                                                                                                                   
                  '_PR':      '_ADP',                                                                                                                                                                                                                                                                    
                  '_S':       '_NOUN',                                                                                                                                                                                                                                                                   
                  '_SPRO':    '_PRON',                                                                                                                                                                                                                                                                   
                  '_UNKN':    '_X',                                                                                                                                                                                                                                                                      
                  '_V':       '_VERB',
                  '':        ''}
    return utags_dict[mystem_prefix]

def get_word_prefix(word):
    lemmas = mystem.analyze(word)
    prefix = ""
    if (isinstance(lemmas, collections.Iterable)) and ('analysis' in lemmas[0]):
        try:
            prefix = "_"+(lemmas[0]['analysis'][0]['gr'].split("=")[0].split(",")[0]) 
        except IndexError:
            prefix = ""
    return convert_prefix2utags(prefix)

def lemmatize_words(word_list):
    processed_word_list = []
    for word in word_list:
        word = mystem.lemmatize(word)[0]
        word_w_prefix = word+get_word_prefix(word)
        processed_word_list.append(word_w_prefix)
    return processed_word_list    

In [10]:
print(lemmatize_words(["люди"]))

['человек_NOUN']


In [11]:
def remove_stopwords(word_list):
        processed_word_list = []
        for word in word_list:
            word = word.lower() # in case they arenet all lower cased
            if word not in stopwords.words("russian") and len(word) > 2:
                processed_word_list.append(word)
        return processed_word_list    

In [12]:
test_tokenized = nltk.word_tokenize(seq_list[1200])
test_clean = remove_stopwords(test_tokenized)

In [13]:
def preprocess_doc(doc_text):
    words_list = remove_stopwords(nltk.word_tokenize(doc_text))
    lemmas_list = lemmatize_words(words_list)
    return " ".join(lemmas_list)

In [17]:
lemmas_doc = preprocess_doc(seq_list[1200])
lemmas_doc

'самый_DET интересный_ADJ вопрос_NOUN который_DET задавать_VERB ученый_NOUN исследователь_NOUN старение_NOUN сам_DET принимать_VERB увеличение_NOUN продолжительность_NOUN жизнь_NOUN узнавать_VERB это_PRON большой_ADJ конференция_NOUN старение_NOUN долголетие_NOUN сразу_ADV видно_ADV современный_ADJ состояние_NOUN дело_NOUN наука_NOUN геропротектор_NOUN услышать_VERB довольно_ADV разный_ADJ ответ_NOUN метформин_NOUN аспирин_NOUN ибупрофный_ADJ витамин_NOUN литий_NOUN витамин_NOUN витамин_NOUN статин_NOUN рапамицин_NOUN далее_ADV многое_PRON оставаться_VERB непонятный_ADJ применение_NOUN какой_DET доза_NOUN учитывать_VERB индивидуальный_ADJ особенность_NOUN работать_VERB клинический_ADJ исследование_NOUN данна_NOUN комбинация_NOUN один_DET научный_ADJ статья_NOUN написать_VERB один_DET другой_DET выходить_VERB писать_VERB пропадать_VERB принимать_VERB другой_DET пожалуй_PART майкл_NOUN шнайдер_NOUN стенфорд_NOUN качественно_ADV исследовать_VERB рассматривать_VERB изменение_NOUN сотня_NOU

In [18]:
from nltk import FreqDist
fd = FreqDist(nltk.word_tokenize(lemmas_doc))
fd.most_common(50)

[('старение_NOUN', 14),
 ('шаг_NOUN', 10),
 ('исследование_NOUN', 8),
 ('все_PRON', 8),
 ('человек_NOUN', 7),
 ('терапия_NOUN', 7),
 ('клинический_ADJ', 6),
 ('сам_DET', 5),
 ('это_PRON', 5),
 ('один_DET', 5),
 ('против_ADP', 5),
 ('вопрос_NOUN', 4),
 ('который_DET', 4),
 ('жизнь_NOUN', 4),
 ('научный_ADJ', 4),
 ('другой_DET', 4),
 ('свой_DET', 4),
 ('результат_NOUN', 4),
 ('открывать_VERB', 4),
 ('организация_NOUN', 4),
 ('пост_NOUN', 4),
 ('самый_DET', 3),
 ('принимать_VERB', 3),
 ('витамин_NOUN', 3),
 ('создавать_VERB', 3),
 ('медицинский_ADJ', 3),
 ('должный_ADJ', 3),
 ('анализ_NOUN', 3),
 ('тип_NOUN', 3),
 ('собственный_ADJ', 3),
 ('проект_NOUN', 3),
 ('голос_NOUN', 3),
 ('запись_NOUN', 3),
 ('долголетие_NOUN', 2),
 ('дело_NOUN', 2),
 ('наука_NOUN', 2),
 ('статин_NOUN', 2),
 ('далее_ADV', 2),
 ('какой_DET', 2),
 ('работать_VERB', 2),
 ('майкл_NOUN', 2),
 ('шнайдер_NOUN', 2),
 ('препарат_NOUN', 2),
 ('продление_NOUN', 2),
 ('процесс_NOUN', 2),
 ('плюс_NOUN', 2),
 ('биомаркер_NOUN',

In [16]:
fout = codecs.open('datasets/golos_io_topics-res-lines.csv', encoding='utf-8', mode='w+')
for item in seq_list:
    if item is not None:
        lemmas_doc = preprocess_doc(item)
        print(lemmas_doc, file=fout, sep="\n")
fout.close()

In [92]:
text = "люди употребляют различные препараты делятся своими впечатлениями . акцент ноотропы , продление жизни . всему этому процессу хватает организованности . собственно этим займемся . прекрасный план . шаг 1. личный медицинский кабинет . должны результаты ваших анализов . сдавали . терапии , которые осуществляли . самое важно , должны результаты диагностики старения . плюс , , описание интервенций против старения . биомаркеры скорости старения далеки совершенств"

In [93]:
from pymystem3 import Mystem
m = Mystem()
lemmas = m.analyze(text)
print(lemmas)

[{'analysis': [{'lex': 'человек', 'gr': 'S,муж,од=им,мн'}], 'text': 'люди'}, {'text': ' '}, {'analysis': [{'lex': 'употреблять', 'gr': 'V,пе=непрош,мн,изъяв,3-л,несов'}], 'text': 'употребляют'}, {'text': ' '}, {'analysis': [{'lex': 'различный', 'gr': 'A=(вин,мн,полн,неод|им,мн,полн)'}], 'text': 'различные'}, {'text': ' '}, {'analysis': [{'lex': 'препарат', 'gr': 'S,муж,неод=(вин,мн|им,мн)'}], 'text': 'препараты'}, {'text': ' '}, {'analysis': [{'lex': 'делиться', 'gr': 'V,несов,нп=непрош,мн,изъяв,3-л'}], 'text': 'делятся'}, {'text': ' '}, {'analysis': [{'lex': 'свой', 'gr': 'APRO=твор,мн'}], 'text': 'своими'}, {'text': ' '}, {'analysis': [{'lex': 'впечатление', 'gr': 'S,сред,неод=твор,мн'}], 'text': 'впечатлениями'}, {'text': ' '}, {'text': '.'}, {'text': ' '}, {'analysis': [{'lex': 'акцент', 'gr': 'S,муж,неод=(вин,ед|им,ед)'}], 'text': 'акцент'}, {'text': ' '}, {'analysis': [{'lex': 'ноотроп', 'qual': 'bastard', 'gr': 'S,муж,неод=(вин,мн|им,мн)'}], 'text': 'ноотропы'}, {'text': ' , '},

In [94]:
lemmas[0]['analysis'][0]['gr'].split(",")[0]

'S'